**Exercise 1.7. (Hedging a long position-multiple periods).** Consider a bank that has a long position in the lookback option of Example 1.2.4. The bank intends to hold this option until expiration and receive the payoff $V_3$. At time zero, the bank has capital $V_0 = 1.376$ tied up in the option and wants to earn the interest rate of $25\%$ on this capital until time three (i.e., without investing any more money, and regardless of how the coin tossing turns out, the bank wants to have

$$\left(\frac{5}{4}\right)^3 \cdot 1.376 = 2.6875$$

at time three, after collecting the payoff from the lookback option at time three). Specify how the bank’s trader should invest in the stock and the money market account to accomplish this.

---

The excercise asks us how to invest in the stock and money market so that the final portfolio value is:

$$\left(\frac{5}{4}\right)^3 \cdot 1.376 = (1 + r)^3 V_0 = 2.6875$$

---

As shown in Example 1.2.4, $V_0 = 1.376$ is the the no-arbitrage price of the derivative security at **time zero**.

In this case, Theorem 1.2.2 holds. Given the formula (1.2.18)

$$X_N(\omega_1 \omega_2 \ldots \omega_N) = V_N(\omega_1 \omega_2 \ldots \omega_N),$$

the payoff $V_1$ of this derivative security at **time one** can be replicated by the corresponding portfolio given by the *wealth equation* formula (1.2.14):

In [1]:
using SymPy
@vars Δ₀ S₁ r V₀ S₀
V₁ = Δ₀*S₁ + (1 + r)*(V₀ - Δ₀*S₀)

S₁⋅Δ₀ + (r + 1)⋅(-S₀⋅Δ₀ + V₀)

which is equivalent to

$$V_1 = \Delta_0 S_1 + (1 + r) V_0 - (1 + r) \Delta_0 S_0.$$

The expression hints that we should cancel

$$-(1 + r) \Delta_0 S_0$$

and

$$\Delta_0 S_1,$$

to get the portfolio

$$\zeta_1 = (1 + r) V_0.$$

To accomplish that, we should:

* short $\Delta_0 S_0$ in stocks and invest the income into the money market at **time zero**; and
* use the money market account to cover the short position $\Delta_0 S_1$ at **time one**.

At **time one** we have the total portfolio value:

In [2]:
ζ₁ = simplify(V₁ + (1 + r)*Δ₀*S₀ - Δ₀*S₁)

V₀⋅(r + 1)

Let's see the replicated payoff of the derivative security at **time two**:

In [3]:
@vars Δ₁ S₂
V₂ = Δ₁*S₂ + (1 + r)*(V₁ - Δ₁*S₁)

S₂⋅Δ₁ + (r + 1)⋅(S₁⋅Δ₀ - S₁⋅Δ₁ + (r + 1)⋅(-S₀⋅Δ₀ + V₀))

It is equivalent to

$$V_2 = \Delta_1 S_2 + (1 + r) [-\Delta_1 S_1 - (1 + r) \Delta_0 S_0 + \Delta_0 S_1] + (1 + r)^2 V_0.$$

The expression hints that we should cancel

$$(1 + r) [-\Delta_1 S_1 - (1 + r) \Delta_0 S_0 + \Delta_0 S_1]$$

and

$$\Delta_1 S_2$$

to get the portfolio

$$\zeta_2 = (1 + r)^2 V_0.$$

To accomplish that, we should:

* short $\Delta_1 S_1$ in stocks and invest the income into the money market and reinvest what is already in the money market account at **time one**; and
* use the money market account to cover the short position $\Delta_1 S_2$ at **time two**.

At **time two** we have the total portfolio value:

In [4]:
ζ₂ = factor(V₂ + (1 + r)*(Δ₁*S₁ + (1 + r)Δ₀*S₀ - Δ₀*S₁) - Δ₁*S₂)

          2
V₀⋅(r + 1) 

The payoff of this strategy can be formulated by:

$$\rho_0 = 0.$$

and

$$\rho_{i + 1} = (1 + r)(\Delta_i S_i + \rho_i) - \Delta_i S_{i + 1}$$

It seems that with this strategy we can obtain the total portfolio value at time $i + 1$:

$$\zeta_{i + 1} = V_{i + 1} + \rho_{i + 1} = (1 + r)^{i + 1} V_0,$$

from which it follows that

$$\rho_{i + 1} = (1 + r)^{i + 1} V_0 - V_{i + 1}.$$

We prove this relationship by induction.

For the base case $i = 0$, we must prove that:

$$\rho_1 = (1 + r)(\Delta_0 S_0 + \rho_0) - \Delta_0 S_1 = (1 + r) V_0 - V_1.$$

Since $\rho_0 = 0$, we define $\rho_1$:

In [5]:
ρ₁ = (1 + r)*(Δ₀*S₀ + 0) - Δ₀*S₁

S₀⋅Δ₀⋅(r + 1) - S₁⋅Δ₀

So we check:

In [6]:
expand(ρ₁) == expand((1 + r)*V₀ - V₁)

true

which proves the base case.

For the inductive step, we assume that

$$\rho_i = (1 + r)^i V_0 - V_i.$$

Using this assumption, and the formulas for $\rho_{i + 1}$ and $V_{i + 1}$ we get the following system of linear equations:

In [7]:
@vars i ρᵢ ρᵢ₊₁ Vᵢ Vᵢ₊₁ Δᵢ Sᵢ Sᵢ₊₁ k
Pᵢ₊₁ = [
    ρᵢ ⩵ ((1 + r)^i)*V₀ - Vᵢ,
    ρᵢ₊₁ ⩵ (1 + r)*(Δᵢ*Sᵢ + ρᵢ) - Δᵢ*Sᵢ₊₁,
    Vᵢ₊₁ ⩵ Δᵢ*Sᵢ₊₁ + (1 + r)*(Vᵢ - Δᵢ*Sᵢ)
]

3-element Array{Sym,1}:
                Eq(ρᵢ, -Vᵢ + V₀*(r + 1)^i)
 Eq(ρᵢ₊₁, -Sᵢ₊₁*Δᵢ + (r + 1)*(Sᵢ*Δᵢ + ρᵢ))
 Eq(Vᵢ₊₁, Sᵢ₊₁*Δᵢ + (r + 1)*(-Sᵢ*Δᵢ + Vᵢ))

We solve for $\rho_i$, $\rho_{i + 1}$ and $\Delta_i$ (we could have chosen either $\Delta_i$, $S_i$ or $S_{i + 1}$, since these terms will be cancelled out of the solution of $\rho_{i + 1}$)

In [8]:
solution = solve(Pᵢ₊₁, [ρᵢ₊₁, ρᵢ, Sᵢ])

Dict{Sym,Sym} with 3 entries:
  Sᵢ   => (Sᵢ₊₁*Δᵢ + Vᵢ*r + Vᵢ - Vᵢ₊₁)/(Δᵢ*(r + 1))
  ρᵢ   => -Vᵢ + V₀*(r + 1)^i
  ρᵢ₊₁ => -Vᵢ₊₁ + V₀*r*(r + 1)^i + V₀*(r + 1)^i

We do some algebraic manipulations to get:

In [9]:
ρᵢ₊₁ = simplify(solution[ρᵢ₊₁] + Vᵢ₊₁) - Vᵢ₊₁

                  i + 1
-Vᵢ₊₁ + V₀⋅(r + 1)     

So we proved that if $\rho_n$ is true, then $\rho_{n + 1}$ is also true. That concludes the proof.

$$\square$$

We can check this result numerically as follows:

As given by Example 1.2.4 and the excercise, we define $S_0$, $r$, $u$, $d$ and $V_0$:

In [10]:
S₀ = 4
r = 1/4
u = 2
d = 1/2
p̃ = q̃ = 1/2
V₀ = 1.376;

We define the Toss enum:

In [11]:
@enum Toss H T

Given $S_0$, $u$ and $d$ we define $S(\omega_1 \omega_2 \ldots \omega_i) = S_i$:

In [12]:
function S(ω::Toss...)
    i = length(ω)
    if i == 0
        return S₀
    else
        Sᵢ₋₁ = S(ω[1 : end - 1]...)
        if ω[i] == H
            return u*Sᵢ₋₁
        else
            return d*Sᵢ₋₁
        end
    end
end;

Given the lookback option payoff

$$V_3 = \max_{0 \leq n \leq 3} S_n - S_3$$

and the formula (1.2.16)

$$V_i = \frac{1}{1 + r} (\tilde p V_{i + 1}(H) + \tilde q V_{i + 1}(T)),$$

we define $V(\omega_1 \omega_2 \ldots \omega_i) = V_i$:

In [13]:
function V(ω::Toss...)
    i = length(ω)
    if i == 3
        S₁ = S(ω[1])
        S₂ = S(ω[1 : 2]...)
        S₃ = S(ω[1 : 3]...)
        # lookback option payoff
        return max(S₀ - S₃, S₁ - S₃, S₂ - S₃, S₃ - S₃)
    elseif i < 3
        # formula (1.2.16)
        return 1/(1 + r)*(p̃*V(ω..., H) + q̃*V(ω..., T))
    else
        error("out of domain parameter")
    end
end;

From the formula (1.2.17)

$$\Delta_i = \frac{V_{i + 1}(H) - V_{i + 1}(T)}{S_{i + 1}(H) - S_{i + 1}(T)},$$

we define $\Delta(\omega_1 \omega_2 \ldots \omega_i) = \Delta_i$:

In [14]:
Δ(ω::Toss...) = (V(ω..., H) - V(ω..., T))/(S(ω..., H) - S(ω..., T));

Given

$$\rho_0 = 0$$

and reindexing the strategy derived in this excercise

$$\rho_{i} = (1 + r)(\Delta_{i - 1} S_{i - 1} + \rho_{i - 1}) - \Delta_{i - 1} S_i.$$

we define $\rho(\omega_1 \omega_2 \ldots \omega_i) = \rho_i$:

In [15]:
function ρ(ω::Toss...)
    i = length(ω)
    if i == 0
        # ρ₀ = 0
        return 0
    else
        Δᵢ₋₁ = Δ(ω[1 : end - 1]...)
        Sᵢ = S(ω...)
        ρᵢ₋₁ = ρ(ω[1 : end - 1]...)
        Sᵢ₋₁ = S(ω[1 : end - 1]...)
        return (1 + r)*(Δᵢ₋₁*Sᵢ₋₁ + ρᵢ₋₁) - Δᵢ₋₁*Sᵢ
    end
end;    

Hence, we get the total portfolio value $\zeta_3$ at **time 3** for every possible outcome in the sample space:

In [16]:
ωᵢ = instances(Toss)
for ω₁ ∈ ωᵢ, ω₂ ∈ ωᵢ, ω₃ ∈ ωᵢ
    V₃ = V(ω₁, ω₂, ω₃)
    ρ₃ = ρ(ω₁, ω₂, ω₃)
    ζ₃ = V₃ + ρ₃
    println("ζ₃($ω₁$ω₂$ω₃) = $ζ₃")
end

ζ₃(HHH) = 2.6875
ζ₃(HHT) = 2.6875
ζ₃(HTH) = 2.6875
ζ₃(HTT) = 2.6875
ζ₃(THH) = 2.6875
ζ₃(THT) = 2.6875
ζ₃(TTH) = 2.6875
ζ₃(TTT) = 2.6875
